In [ ]:
Tuan Pham, Victoria Hall, Alexy Correa, Sahil Jain  

# Final Project

One of the most important (and usually time consuming!) steps in the data science process is data pre-processing. 

Data prep includes:
* Data exploration  
* Data cleaning
* Feature engineering

In this assignment, you will do a little of each of these things. 